In [1]:

import torch
from torch import optim
from torch.autograd import Variable
import numpy as np
import os
import math
from torch import nn
import torchvision

from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
import matplotlib.pyplot as plt  
import itertools
import random
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
torch.cuda.is_available()

True

## DataLoader

In [3]:
### handle the dataset
class TorchDataset(Dataset):
    def __init__(self, trs_file, label_file, trace_num, trace_offset, trace_length):
        self.trs_file = trs_file
        self.label_file = label_file
        self.trace_num = trace_num
        self.trace_offset = trace_offset
        self.trace_length = trace_length
        self.ToTensor = transforms.ToTensor()
    def __getitem__(self, i):
        index = i % self.trace_num
        trace = self.trs_file[index,:]
        label = self.label_file[index]
        trace = trace[self.trace_offset:self.trace_offset+self.trace_length]
        trace = np.reshape(trace,(1,-1))
        trace = self.ToTensor(trace)
        trace = np.reshape(trace, (1,-1))
        label = torch.tensor(label, dtype=torch.long) 
        return trace.float(), label
    def __len__(self):
        return self.trace_num
    
### data loader for training
def load_training(batch_size, kwargs):
    data = TorchDataset(**kwargs)
    train_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=1, pin_memory=True)
    return train_loader

### data loader for testing
def load_testing(batch_size, kwargs):
    data = TorchDataset(**kwargs)
    test_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=False, drop_last=True, num_workers=1, pin_memory=True)
    return test_loader

## Arrays and Functions

In [4]:
Sbox = [99, 124, 119, 123, 242, 107, 111, 197, 48, 1, 103, 43, 254, 215, 171, 118, 202, 130, 201, 125, 250, 89, 71,
        240, 173, 212, 162, 175, 156, 164, 114, 192, 183, 253, 147, 38, 54, 63, 247, 204, 52, 165, 229, 241, 113, 216,
        49, 21, 4, 199, 35, 195, 24, 150, 5, 154, 7, 18, 128, 226, 235, 39, 178, 117, 9, 131, 44, 26, 27, 110, 90, 160,
        82, 59, 214, 179, 41, 227, 47, 132, 83, 209, 0, 237, 32, 252, 177, 91, 106, 203, 190, 57, 74, 76, 88, 207, 208,
        239, 170, 251, 67, 77, 51, 133, 69, 249, 2, 127, 80, 60, 159, 168, 81, 163, 64, 143, 146, 157, 56, 245, 188,
        182, 218, 33, 16, 255, 243, 210, 205, 12, 19, 236, 95, 151, 68, 23, 196, 167, 126, 61, 100, 93, 25, 115, 96,
        129, 79, 220, 34, 42, 144, 136, 70, 238, 184, 20, 222, 94, 11, 219, 224, 50, 58, 10, 73, 6, 36, 92, 194, 211,
        172, 98, 145, 149, 228, 121, 231, 200, 55, 109, 141, 213, 78, 169, 108, 86, 244, 234, 101, 122, 174, 8, 186,
        120, 37, 46, 28, 166, 180, 198, 232, 221, 116, 31, 75, 189, 139, 138, 112, 62, 181, 102, 72, 3, 246, 14, 97,
        53, 87, 185, 134, 193, 29, 158, 225, 248, 152, 17, 105, 217, 142, 148, 155, 30, 135, 233, 206, 85, 40, 223, 140,
        161, 137, 13, 191, 230, 66, 104, 65, 153, 45, 15, 176, 84, 187, 22]

HW_byte = [0, 1, 1, 2, 1, 2, 2, 3, 1, 2, 2, 3, 2, 3, 3, 4, 1, 2, 2, 3, 2, 3, 3, 4, 2, 3, 3, 4, 3, 4, 4, 5, 1, 2, 2,
            3, 2, 3, 3, 4, 2, 3, 3, 4, 3, 4, 4, 5, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 1, 2, 2, 3, 2, 3,
            3, 4, 2, 3, 3, 4, 3, 4, 4, 5, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 2, 3, 3, 4, 3, 4, 4, 5, 3,
            4, 4, 5, 4, 5, 5, 6, 3, 4, 4, 5, 4, 5, 5, 6, 4, 5, 5, 6, 5, 6, 6, 7, 1, 2, 2, 3, 2, 3, 3, 4, 2, 3, 3, 4,
            3, 4, 4, 5, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5,
            6, 3, 4, 4, 5, 4, 5, 5, 6, 4, 5, 5, 6, 5, 6, 6, 7, 2, 3, 3, 4, 3, 4, 4, 5, 3, 4, 4, 5, 4, 5, 5, 6, 3, 4,
            4, 5, 4, 5, 5, 6, 4, 5, 5, 6, 5, 6, 6, 7, 3, 4, 4, 5, 4, 5, 5, 6, 4, 5, 5, 6, 5, 6, 6, 7, 4, 5, 5, 6, 5,
            6, 6, 7, 5, 6, 6, 7, 6, 7, 7, 8]

def train(epoch, model, freeze_BN=False):
    """
    - epoch : the current epoch
    - model : the current model  
    - freeze_BN : whether to freeze batch normalization layers
    """
    if freeze_BN:
        model.eval()  # enter eval mode to freeze batch normalization layers
    else:
        model.train()  # enter training mode 
    # get the number of batches
    num_iter = len(source_train_loader)
    clf_criterion = nn.CrossEntropyLoss()
    # train on each batch of data
    for i, (source_data, source_label) in enumerate(source_train_loader, 1):
        if cuda:
            source_data, source_label = source_data.cuda(), source_label.cuda()
        source_data, source_label = Variable(source_data), Variable(source_label)
        optimizer.zero_grad()
        source_preds = model(source_data)
        preds = source_preds.data.max(1, keepdim=True)[1]
        correct_batch = preds.eq(source_label.data.view_as(preds)).sum()
        loss = clf_criterion(source_preds, source_label)
        # optimize the cross-entropy loss
        loss.backward()
        optimizer.step()
        if i % log_interval == 0:
            print('Train Epoch {}: [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAcc: {:.6f}%'.format(
                epoch, i * len(source_data), len(source_train_loader) * batch_size,
                100. * i / len(source_train_loader), loss.data, 100. * correct_batch / len(source_data)))

            
### validation 
def validation(model):
    # enter evaluation mode
    model.eval()
    valid_loss = 0
    # the number of correct prediction
    correct_valid = 0
    clf_criterion = nn.CrossEntropyLoss()
    for data, label in source_valid_loader:
        if cuda:
            data, label = data.cuda(), label.cuda()
        data, label = Variable(data), Variable(label)
        valid_preds = model(data)
        # sum up batch loss
        valid_loss += clf_criterion(valid_preds, label) 
        # get the index of the max probability
        pred = valid_preds.data.max(1)[1] 
        # get the number of correct prediction
        correct_valid += pred.eq(label.data.view_as(pred)).cpu().sum()
    valid_loss /= len(source_valid_loader)
    valid_acc = 100. * correct_valid / len(source_valid_loader.dataset)
    print('Validation: loss: {:.4f}, accuracy: {}/{} ({:.6f}%)'.format(
        valid_loss.data, correct_valid, len(source_valid_loader.dataset),
        valid_acc))
    return valid_loss, valid_acc

### test/attack
def test(model, device_id, disp_GE=True, model_flag='pretrained'):
    """
    - model : the current model 
    - device_id : id of the tested device
    - disp_GE : whether to attack/calculate guessing entropy (GE)
    - model_flag : a string for naming GE result
    """
    # enter evaluation mode
    model.eval()
    test_loss = 0
    # the number of correct prediction
    correct = 0
    epoch = 0
    clf_criterion = nn.CrossEntropyLoss()
    if device_id == source_device_id: # attack on the source domain
        test_num = source_test_num
        test_loader = source_test_loader
        real_key = real_key_01
    else: # attack on the target domain
        test_num = target_test_num
        test_loader = target_test_loader
        real_key = real_key_02
    # Initialize the prediction and label lists(tensors)
    predlist=torch.zeros(0,dtype=torch.long, device='cpu')
    lbllist=torch.zeros(0,dtype=torch.long, device='cpu')
    test_preds_all = torch.zeros((test_num, class_num), dtype=torch.float, device='cpu')
    for data, label in test_loader:
        if cuda:
            data, label = data.cuda(), label.cuda()
        data, label = Variable(data), Variable(label)
        test_preds = model(data)
        # sum up batch loss
        test_loss += clf_criterion(test_preds, label) 
        # get the index of the max probability
        pred = test_preds.data.max(1)[1]
        # get the softmax results for attack/showing guessing entropy
        softmax = nn.Softmax(dim=1)
        test_preds_all[epoch*batch_size:(epoch+1)*batch_size, :] =softmax(test_preds)
        # get the predictions (predlist) and real labels (lbllist) for showing confusion matrix
        predlist=torch.cat([predlist,pred.view(-1).cpu()])
        lbllist=torch.cat([lbllist,label.view(-1).cpu()])
        # get the number of correct prediction
        correct += pred.eq(label.data.view_as(pred)).cpu().sum()
        epoch += 1
    test_loss /= len(test_loader)
    print('Target test loss: {:.4f}, Target test accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss.data, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    # get the confusion matrix
    confusion_mat = confusion_matrix(lbllist.numpy(), predlist.numpy())
    # show the confusion matrix
    plot_sonfusion_matrix(confusion_mat, classes = range(class_num))
    # show the guessing entropy and success rate
    if disp_GE:
        plot_guessing_entropy(test_preds_all.numpy(), real_key, device_id, model_flag)

### kernel function
def guassian_kernel(source, target, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
    """
    - source : source data
    - target : target data
    - kernel_mul : multiplicative step of bandwidth (sigma)
    - kernel_num : the number of guassian kernels
    - fix_sigma : use a fix value of bandwidth
    """
    n_samples = int(source.size()[0])+int(target.size()[0])
    total = torch.cat([source, target], dim=0)
    total0 = total.unsqueeze(0).expand(int(total.size(0)), \
                                       int(total.size(0)), \
                                       int(total.size(1)))
    total1 = total.unsqueeze(1).expand(int(total.size(0)), \
                                       int(total.size(0)), \
                                       int(total.size(1)))
    # |x-y|
    L2_distance = ((total0-total1)**2).sum(2) 
    
    # bandwidth
    if fix_sigma:
        bandwidth = fix_sigma
    else:
        bandwidth = torch.sum(L2_distance.data) / (n_samples**2-n_samples)
    # take the current bandwidth as the median value, and get a list of bandwidths (for example, when bandwidth is 1, we get [0.25,0.5,1,2,4]). 
    bandwidth /= kernel_mul ** (kernel_num // 2)
    bandwidth_list = [bandwidth * (kernel_mul**i) for i in range(kernel_num)]

    # exp(-|x-y|/bandwidth)
    kernel_val = [torch.exp(-L2_distance / bandwidth_temp) for \
                  bandwidth_temp in bandwidth_list]

    # return the final kernel matrix
    return sum(kernel_val)

### MMD loss function based on guassian kernels
def mmd_rbf(source, target, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
    """
    - source : source data
    - target : target data
    - kernel_mul : multiplicative step of bandwidth (sigma)
    - kernel_num : the number of guassian kernels
    - fix_sigma : use a fix value of bandwidth
    """
    loss = 0.0
    batch_size = int(source.size()[0])
    kernels = guassian_kernel(source, target,kernel_mul=kernel_mul,kernel_num=kernel_num, fix_sigma=fix_sigma)
    XX = kernels[:batch_size, :batch_size] # Source<->Source
    YY = kernels[batch_size:, batch_size:] # Target<->Target
    XY = kernels[:batch_size, batch_size:] # Source<->Target
    YX = kernels[batch_size:, :batch_size] # Target<->Source
    loss = torch.mean(XX + YY - XY -YX)
    return loss

### show the guessing entropy and success rate
def plot_guessing_entropy(preds, real_key, device_id, model_flag):
    """
    - preds : the probability for each class (n*256 for a byte, n*9 for Hamming weight)
    - real_key : the key of the target device
    - device_id : id of the target device
    - model_flag : a string for naming GE result
    """
    # GE/SR is averaged over 100 attacks 
    num_averaged = 1
    # max trace num for attack
    trace_num_max = 10000
    guessing_entropy = np.zeros((num_averaged, trace_num_max))
    success_flag = np.zeros((num_averaged, trace_num_max))
    

    plaintext = plaintexts_source
    # attack multiples times for average
    for time in range(num_averaged):
        # select the attack traces randomly
        random_index = list(range(plaintext.shape[0]))
        random.shuffle(random_index)
        random_index = random_index[0:trace_num_max]
        # initialize score matrix
        score_mat = np.zeros((trace_num_max, 256))
        for key_guess in range(0, 256):
            for i in range(0, trace_num_max):
                initialState = plaintext[random_index[i]] ^ key_guess
                sout = Sbox[int(initialState)]
                if labeling_method == 'identity':
                    label = sout
                elif labeling_method == 'hw':
                    label = HW_byte[sout]
                score_mat[i, key_guess] = preds[random_index[i], label]
        score_mat = np.log(score_mat + 1e-40)
        for i in range(0, trace_num_max):
            log_likelihood = np.sum(score_mat[0:i+1,:], axis=0)
            ranked = np.argsort(log_likelihood)[::-1]
            guessing_entropy[time,i] =  list(ranked).index(real_key)
            if list(ranked).index(real_key) == 0:
                    success_flag[time, i] = 1
    guessing_entropy = np.mean(guessing_entropy,axis=0)
    plt.figure(figsize=(12,4))
    plt.subplot(1, 2, 1)
    p1, = plt.plot(guessing_entropy[0:trace_num_max],color='red')
    plt.xlabel('Number of trace')
    plt.ylabel('Guessing entropy')
    #np.save('./results/entropy_'+ labeling_method + '_{}_to_{}_'.format(source_device_id, device_id) + model_flag, guessing_entropy)
    plt.subplot(1, 2, 2)       
    success_flag = np.sum(success_flag, axis=0)
    success_rate = success_flag/num_averaged 
    p2, = plt.plot(success_rate[0:trace_num_max], color='red')
    plt.xlabel('Number of trace')
    plt.ylabel('Success rate')
    plt.show()
    #np.save('./results/success_rate_' + labeling_method + '_{}_to_{}_'.format(source_device_id, device_id) + model_flag, success_rate)

### show the confusion matrix 
def plot_sonfusion_matrix(cm, classes, normalize=False, title='Confusion matrix',cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float')/cm.sum(axis=1)[:,np.newaxis]
    thresh = cm.max()/2.0
    for i,j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j,i,cm[i,j], horizontalalignment='center',color='white' if cm[i,j] > thresh else 'black')   
    plt.ylim((len(classes)-0.5, -0.5))
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predict label')
    plt.show()

## Setups

In [10]:
model_op='L11'
source_device_id = 1
target_device_id = 1
real_key_01 = 0x15 # key of the source domain
real_key_02 = 0x15 # key of the target domain
lambda_ = 0.1 # Penalty coefficient
labeling_method = 'hw' # labeling of trace
preprocess = 'horizontal_standardization' # preprocess method
batch_size = 250
lr = 0.001 # learning rate
log_interval = 20 # epoch interval to log training information
train_num = 135000
valid_num = 5000
source_test_num = 10000
trace_offset = 0
trace_length = 1500
source_file_path = './Data/ourdata/xmega/RD/T1/'
no_cuda =False
cuda = not no_cuda and torch.cuda.is_available()
seed = 8
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)
if labeling_method == 'identity':
    class_num = 256
elif labeling_method == 'hw':
    class_num = 9
    
# to load traces and labels
X_train_source = np.load(source_file_path + 'X_train.npy')
Y_train_source = np.load(source_file_path + 'Y_train.npy')
X_attack_source = np.load(source_file_path + 'X_attack.npy')
Y_attack_source = np.load(source_file_path + 'Y_attack.npy')

# to load plaintexts
plaintexts_source = np.load(source_file_path + 'plaintexts_attack.npy')

# preprocess of traces
if preprocess == 'horizontal_standardization':
    mn = np.repeat(np.mean(X_train_source, axis=1, keepdims=True), X_train_source.shape[1], axis=1)
    std = np.repeat(np.std(X_train_source, axis=1, keepdims=True), X_train_source.shape[1], axis=1)
    X_train_source = (X_train_source - mn)/std

    mn = np.repeat(np.mean(X_attack_source, axis=1, keepdims=True), X_attack_source.shape[1], axis=1)
    std = np.repeat(np.std(X_attack_source, axis=1, keepdims=True), X_attack_source.shape[1], axis=1)
    X_attack_source = (X_attack_source - mn)/std
    
elif preprocess == 'horizontal_scaling':
    scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1)).fit(X_train_source.T)
    X_train_source = scaler.transform(X_train_source.T).T

    scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1)).fit(X_attack_source.T)
    X_attack_source = scaler.transform(X_attack_source.T).T
    
# parameters of data loader
kwargs_source_train = {
        'trs_file': X_train_source[0:train_num,:],
        'label_file': Y_train_source[0:train_num],
        'trace_num':train_num,
        'trace_offset':trace_offset,
        'trace_length':trace_length,
}
kwargs_source_valid = {
        'trs_file': X_train_source[train_num:train_num+valid_num,:],
        'label_file': Y_train_source[train_num:train_num+valid_num],
        'trace_num':valid_num,
        'trace_offset':trace_offset,
        'trace_length':trace_length,
}
kwargs_source_test = {
        'trs_file': X_attack_source,
        'label_file': Y_attack_source,
        'trace_num':source_test_num,
        'trace_offset':trace_offset,
        'trace_length':trace_length,
}

source_train_loader = load_training(batch_size, kwargs_source_train)
source_valid_loader = load_training(batch_size, kwargs_source_valid)
source_test_loader = load_testing(batch_size, kwargs_source_test)
print('Load data complete!')

Load data complete!


## Models

In [6]:
### the pre-trained model
class Net(nn.Module):
    def __init__(self, num_classes=class_num):
        super(Net, self).__init__()
        # the encoder part
        self.features = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=1),
            nn.SELU(),
            nn.AvgPool1d(kernel_size=2, stride=2),
            nn.Conv1d(16, 32, kernel_size=50),
            nn.SELU(),
            nn.AvgPool1d(kernel_size=50, stride=50),
            nn.Conv1d(32, 64, kernel_size=3),
            nn.SELU(),
            nn.AvgPool1d(kernel_size=2, stride=2),
            nn.Flatten()
        )
        # the fully-connected layer 1
        self.classifier_1 = nn.Sequential(
            nn.Linear(384, 20),
            nn.ReLU(inplace=True),
        )
        # the output layer
        self.final_classifier = nn.Sequential(
            nn.Linear(20, num_classes)
        )

    # how the network runs
    def forward(self, input):
        x = self.features(input)
        x = x.view(x.size(0), -1)
        x = self.classifier_1(x)
        output = self.final_classifier(x)
        return output


In [7]:
# create a network
model = Net(num_classes=class_num)
print('Construct model complete')
if cuda:
    model.cuda()
# initialize a big enough loss
min_loss = 1000
# load the pre-trained model
#checkpoint = torch.load('./models/new{}.pth'.format(source_device_id))
optimizer = optim.Adam([
        {'params': model.features.parameters()},
        {'params': model.classifier_1.parameters()},
        {'params': model.final_classifier.parameters()}
    ], lr=lr)
# restore the optimizer state
#optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Usage:
for epoch in range(1, 150 + 1):
    print(f'Train Epoch {epoch}:')
    train(epoch, model)

Construct model complete
Train Epoch 1:
Train Epoch 1: [5000/135000 (4%)]	Loss: 1.893665	Acc: 28.400002%
Train Epoch 1: [10000/135000 (7%)]	Loss: 1.739390	Acc: 25.200001%
Train Epoch 1: [15000/135000 (11%)]	Loss: 1.803112	Acc: 25.200001%
Train Epoch 1: [20000/135000 (15%)]	Loss: 1.739239	Acc: 29.600002%
Train Epoch 1: [25000/135000 (19%)]	Loss: 1.724337	Acc: 30.000002%
Train Epoch 1: [30000/135000 (22%)]	Loss: 1.849668	Acc: 25.200001%
Train Epoch 1: [35000/135000 (26%)]	Loss: 1.804479	Acc: 24.000002%
Train Epoch 1: [40000/135000 (30%)]	Loss: 1.758863	Acc: 26.000002%
Train Epoch 1: [45000/135000 (33%)]	Loss: 1.781791	Acc: 26.800001%
Train Epoch 1: [50000/135000 (37%)]	Loss: 1.759945	Acc: 24.800001%
Train Epoch 1: [55000/135000 (41%)]	Loss: 1.785782	Acc: 29.200001%
Train Epoch 1: [60000/135000 (44%)]	Loss: 1.783183	Acc: 26.000002%
Train Epoch 1: [65000/135000 (48%)]	Loss: 1.776527	Acc: 29.600002%
Train Epoch 1: [70000/135000 (52%)]	Loss: 1.797303	Acc: 28.800001%
Train Epoch 1: [75000/135

Train Epoch 5: [70000/135000 (52%)]	Loss: 1.755420	Acc: 28.400002%
Train Epoch 5: [75000/135000 (56%)]	Loss: 1.871524	Acc: 23.600000%
Train Epoch 5: [80000/135000 (59%)]	Loss: 1.813910	Acc: 22.000002%
Train Epoch 5: [85000/135000 (63%)]	Loss: 1.758759	Acc: 27.600000%
Train Epoch 5: [90000/135000 (67%)]	Loss: 1.760730	Acc: 27.600000%
Train Epoch 5: [95000/135000 (70%)]	Loss: 1.826175	Acc: 27.200001%
Train Epoch 5: [100000/135000 (74%)]	Loss: 1.729702	Acc: 32.000000%
Train Epoch 5: [105000/135000 (78%)]	Loss: 1.803289	Acc: 22.400002%
Train Epoch 5: [110000/135000 (81%)]	Loss: 1.770281	Acc: 24.400002%
Train Epoch 5: [115000/135000 (85%)]	Loss: 1.766029	Acc: 31.200001%
Train Epoch 5: [120000/135000 (89%)]	Loss: 1.707260	Acc: 30.800001%
Train Epoch 5: [125000/135000 (93%)]	Loss: 1.772091	Acc: 23.600000%
Train Epoch 5: [130000/135000 (96%)]	Loss: 1.708945	Acc: 30.800001%
Train Epoch 5: [135000/135000 (100%)]	Loss: 1.756502	Acc: 28.800001%
Train Epoch 6:
Train Epoch 6: [5000/135000 (4%)]	Loss

Train Epoch 9: [135000/135000 (100%)]	Loss: 1.336456	Acc: 43.600002%
Train Epoch 10:
Train Epoch 10: [5000/135000 (4%)]	Loss: 1.332557	Acc: 42.000004%
Train Epoch 10: [10000/135000 (7%)]	Loss: 1.346785	Acc: 36.000000%
Train Epoch 10: [15000/135000 (11%)]	Loss: 1.317939	Acc: 45.200001%
Train Epoch 10: [20000/135000 (15%)]	Loss: 1.321066	Acc: 42.000004%
Train Epoch 10: [25000/135000 (19%)]	Loss: 1.359895	Acc: 40.800003%
Train Epoch 10: [30000/135000 (22%)]	Loss: 1.271833	Acc: 44.400002%
Train Epoch 10: [35000/135000 (26%)]	Loss: 1.311392	Acc: 43.600002%
Train Epoch 10: [40000/135000 (30%)]	Loss: 1.362924	Acc: 41.600002%
Train Epoch 10: [45000/135000 (33%)]	Loss: 1.281648	Acc: 42.800003%
Train Epoch 10: [50000/135000 (37%)]	Loss: 1.333503	Acc: 40.400002%
Train Epoch 10: [55000/135000 (41%)]	Loss: 1.334803	Acc: 41.200001%
Train Epoch 10: [60000/135000 (44%)]	Loss: 1.292932	Acc: 42.000004%
Train Epoch 10: [65000/135000 (48%)]	Loss: 1.318617	Acc: 41.200001%
Train Epoch 10: [70000/135000 (52%

Train Epoch 14: [55000/135000 (41%)]	Loss: 1.218451	Acc: 46.400002%
Train Epoch 14: [60000/135000 (44%)]	Loss: 1.402051	Acc: 40.400002%
Train Epoch 14: [65000/135000 (48%)]	Loss: 1.264381	Acc: 42.000004%
Train Epoch 14: [70000/135000 (52%)]	Loss: 1.286563	Acc: 42.800003%
Train Epoch 14: [75000/135000 (56%)]	Loss: 1.254342	Acc: 46.400002%
Train Epoch 14: [80000/135000 (59%)]	Loss: 1.290150	Acc: 45.600002%
Train Epoch 14: [85000/135000 (63%)]	Loss: 1.226739	Acc: 44.000004%
Train Epoch 14: [90000/135000 (67%)]	Loss: 1.312278	Acc: 45.600002%
Train Epoch 14: [95000/135000 (70%)]	Loss: 1.220222	Acc: 50.000004%
Train Epoch 14: [100000/135000 (74%)]	Loss: 1.193526	Acc: 49.600002%
Train Epoch 14: [105000/135000 (78%)]	Loss: 1.206301	Acc: 46.000004%
Train Epoch 14: [110000/135000 (81%)]	Loss: 1.352902	Acc: 40.000000%
Train Epoch 14: [115000/135000 (85%)]	Loss: 1.296282	Acc: 44.800003%
Train Epoch 14: [120000/135000 (89%)]	Loss: 1.252198	Acc: 45.600002%
Train Epoch 14: [125000/135000 (93%)]	Loss:

Train Epoch 18: [115000/135000 (85%)]	Loss: 1.320967	Acc: 41.200001%
Train Epoch 18: [120000/135000 (89%)]	Loss: 1.193659	Acc: 47.600002%
Train Epoch 18: [125000/135000 (93%)]	Loss: 1.166833	Acc: 48.800003%
Train Epoch 18: [130000/135000 (96%)]	Loss: 1.213669	Acc: 50.400002%
Train Epoch 18: [135000/135000 (100%)]	Loss: 1.222447	Acc: 46.800003%
Train Epoch 19:
Train Epoch 19: [5000/135000 (4%)]	Loss: 1.242887	Acc: 43.200001%
Train Epoch 19: [10000/135000 (7%)]	Loss: 1.206785	Acc: 52.000004%
Train Epoch 19: [15000/135000 (11%)]	Loss: 1.200082	Acc: 46.800003%
Train Epoch 19: [20000/135000 (15%)]	Loss: 1.222336	Acc: 43.600002%
Train Epoch 19: [25000/135000 (19%)]	Loss: 1.265685	Acc: 49.200001%
Train Epoch 19: [30000/135000 (22%)]	Loss: 1.206056	Acc: 51.200001%
Train Epoch 19: [35000/135000 (26%)]	Loss: 1.143249	Acc: 49.600002%
Train Epoch 19: [40000/135000 (30%)]	Loss: 1.263797	Acc: 46.800003%
Train Epoch 19: [45000/135000 (33%)]	Loss: 1.178934	Acc: 42.400002%
Train Epoch 19: [50000/135000

...

Train Epoch 133: [70000/135000 (52%)]	Loss: 1.145227	Acc: 49.600002%
Train Epoch 133: [75000/135000 (56%)]	Loss: 1.121325	Acc: 50.400002%
Train Epoch 133: [80000/135000 (59%)]	Loss: 1.072933	Acc: 56.000004%
Train Epoch 133: [85000/135000 (63%)]	Loss: 1.078147	Acc: 53.200001%
Train Epoch 133: [90000/135000 (67%)]	Loss: 1.052753	Acc: 55.200001%
Train Epoch 133: [95000/135000 (70%)]	Loss: 1.035435	Acc: 56.400002%
Train Epoch 133: [100000/135000 (74%)]	Loss: 1.107393	Acc: 51.200001%
Train Epoch 133: [105000/135000 (78%)]	Loss: 1.049424	Acc: 54.400002%
Train Epoch 133: [110000/135000 (81%)]	Loss: 1.044389	Acc: 60.000004%
Train Epoch 133: [115000/135000 (85%)]	Loss: 1.057714	Acc: 53.600002%
Train Epoch 133: [120000/135000 (89%)]	Loss: 1.068387	Acc: 54.400002%
Train Epoch 133: [125000/135000 (93%)]	Loss: 0.990265	Acc: 57.200005%
Train Epoch 133: [130000/135000 (96%)]	Loss: 1.089440	Acc: 52.000004%
Train Epoch 133: [135000/135000 (100%)]	Loss: 1.061029	Acc: 56.000004%
Train Epoch 134:
Train Ep

Train Epoch 137: [120000/135000 (89%)]	Loss: 1.103729	Acc: 56.400002%
Train Epoch 137: [125000/135000 (93%)]	Loss: 1.107509	Acc: 59.200005%
Train Epoch 137: [130000/135000 (96%)]	Loss: 1.126145	Acc: 56.000004%
Train Epoch 137: [135000/135000 (100%)]	Loss: 1.115398	Acc: 51.200001%
Train Epoch 138:
Train Epoch 138: [5000/135000 (4%)]	Loss: 1.060271	Acc: 52.000004%
Train Epoch 138: [10000/135000 (7%)]	Loss: 1.049529	Acc: 58.000004%
Train Epoch 138: [15000/135000 (11%)]	Loss: 1.013820	Acc: 56.800003%
Train Epoch 138: [20000/135000 (15%)]	Loss: 1.093754	Acc: 53.200001%
Train Epoch 138: [25000/135000 (19%)]	Loss: 1.100221	Acc: 55.200001%
Train Epoch 138: [30000/135000 (22%)]	Loss: 1.032753	Acc: 55.600002%
Train Epoch 138: [35000/135000 (26%)]	Loss: 1.005755	Acc: 58.400002%
Train Epoch 138: [40000/135000 (30%)]	Loss: 1.037143	Acc: 54.400002%
Train Epoch 138: [45000/135000 (33%)]	Loss: 1.026739	Acc: 54.400002%
Train Epoch 138: [50000/135000 (37%)]	Loss: 1.017193	Acc: 56.400002%
Train Epoch 138

Train Epoch 142: [35000/135000 (26%)]	Loss: 1.041029	Acc: 54.800003%
Train Epoch 142: [40000/135000 (30%)]	Loss: 1.171505	Acc: 49.600002%
Train Epoch 142: [45000/135000 (33%)]	Loss: 1.068112	Acc: 56.400002%
Train Epoch 142: [50000/135000 (37%)]	Loss: 1.108279	Acc: 52.800003%
Train Epoch 142: [55000/135000 (41%)]	Loss: 1.126804	Acc: 50.000004%
Train Epoch 142: [60000/135000 (44%)]	Loss: 1.035633	Acc: 53.600002%
Train Epoch 142: [65000/135000 (48%)]	Loss: 0.993282	Acc: 56.400002%
Train Epoch 142: [70000/135000 (52%)]	Loss: 1.034479	Acc: 57.600002%
Train Epoch 142: [75000/135000 (56%)]	Loss: 1.061258	Acc: 57.600002%
Train Epoch 142: [80000/135000 (59%)]	Loss: 1.054474	Acc: 55.200001%
Train Epoch 142: [85000/135000 (63%)]	Loss: 0.944205	Acc: 59.600002%
Train Epoch 142: [90000/135000 (67%)]	Loss: 1.139253	Acc: 49.600002%
Train Epoch 142: [95000/135000 (70%)]	Loss: 1.076311	Acc: 56.400002%
Train Epoch 142: [100000/135000 (74%)]	Loss: 1.118559	Acc: 47.200001%
Train Epoch 142: [105000/135000 (

Train Epoch 146: [85000/135000 (63%)]	Loss: 1.043413	Acc: 54.000004%
Train Epoch 146: [90000/135000 (67%)]	Loss: 1.035215	Acc: 56.800003%
Train Epoch 146: [95000/135000 (70%)]	Loss: 1.017076	Acc: 56.000004%
Train Epoch 146: [100000/135000 (74%)]	Loss: 1.049445	Acc: 52.800003%
Train Epoch 146: [105000/135000 (78%)]	Loss: 1.109297	Acc: 54.400002%
Train Epoch 146: [110000/135000 (81%)]	Loss: 1.058034	Acc: 55.200001%
Train Epoch 146: [115000/135000 (85%)]	Loss: 1.111229	Acc: 52.800003%
Train Epoch 146: [120000/135000 (89%)]	Loss: 0.987900	Acc: 59.600002%
Train Epoch 146: [125000/135000 (93%)]	Loss: 1.062162	Acc: 55.600002%
Train Epoch 146: [130000/135000 (96%)]	Loss: 1.125625	Acc: 52.800003%
Train Epoch 146: [135000/135000 (100%)]	Loss: 1.012366	Acc: 57.200005%
Train Epoch 147:
Train Epoch 147: [5000/135000 (4%)]	Loss: 1.004328	Acc: 54.000004%
Train Epoch 147: [10000/135000 (7%)]	Loss: 1.017560	Acc: 58.400002%
Train Epoch 147: [15000/135000 (11%)]	Loss: 1.116645	Acc: 55.600002%
Train Epoch

Train Epoch 150: [135000/135000 (100%)]	Loss: 0.971043	Acc: 56.000004%


In [9]:
# evaluate the fine-tuned model on source and target domain
with torch.no_grad():
    print('Result on source device:')
    test(model, source_device_id, model_flag='finetuned_source')


Result on source device:


RuntimeError: mat1 and mat2 shapes cannot be multiplied (250x320 and 384x20)

In [ ]:
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
    }, './models/xmega/newmodel{}.pth'.format(model_op))
